# LLM Compiler Agent

Full credits to the [source repo for LLMCompiler](https://github.com/SqueezeAILab/LLMCompiler).

A lot of our implementation was lifted from this repo (and adapted with LlamaIndex modules).

LLM Compiler agent notebook

In [1]:
# Phoenix can display in real time the traces automatically
# collected from your LlamaIndex application.
import phoenix as px

# Look for a URL in the output to open the App in a browser.
px.launch_app()
# The App is initially empty, but as you proceed with the steps below,
# traces will appear automatically as your LlamaIndex application runs.

import llama_index

llama_index.set_global_handler("arize_phoenix")

# Run all of your LlamaIndex applications as usual and traces
# will be collected and displayed in Phoenix.

🌍 To view the Phoenix app in your browser, visit http://127.0.0.1:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import json
from typing import Sequence, List

from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [4]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

tools = [multiply_tool, add_tool]

In [11]:
from llama_index.agent import AgentRunner
from llama_index.agent.llm_compiler.step import LLMCompilerAgentWorker

In [12]:
llm = OpenAI(model="gpt-4")

In [13]:
callback_manager = llm.callback_manager

In [14]:
agent_worker = LLMCompilerAgentWorker.from_tools(
    tools,
    llm=llm,
    verbose=True,
    callback_manager=callback_manager
)
agent = AgentRunner(agent_worker, callback_manager=callback_manager)

PREFIX: Given a user query, create a plan to solve it with the utmost parallelizability. Each plan should comprise an action from the following 3 types:
1. multiply(a: int, b: int) -> int
Multiple two integers and returns the result integer
2. add(a: int, b: int) -> int
Add two integers and returns the result integer
3. join():
 - Collects and combines results from prior actions.
 - A LLM agent is called upon invoking join to either finalize the user query or wait until the plans are executed.
 - join should always be the last action in the plan, and will be called in two scenarios:
   (a) if the answer can be determined by gathering the outputs from tasks to generate the final response.
   (b) if the answer cannot be determined in the planning phase before you execute the plans. 

Guidelines:
 - Each action described above contains input/output types and description.
    - You must strictly adhere to the input and output types for each action.
    - The action descriptions contain the

In [15]:
response = agent.chat("What is (121 * 3) + 42?")

> Running step db4d859b-6ef4-4dd1-bd1e-6a2c42a0b4a8 for task d966bb3d-95c1-4396-a187-529d394faad0.
> Step count: 0
> Plan: 1. multiply(121, 3)
2. add($1, 42)
3. join()<END_OF_PLAN>
Ran task: multiply. Observation: 363
Ran task: add. Observation: 405
Ran task: join. Observation: None


In [9]:
response

AgentChatResponse(response='405', sources=[], source_nodes=[])